In [1]:
import ctypes
import logging
import multiprocessing as mp

In [2]:
from contextlib import closing

In [3]:
import numpy as np

In [4]:
info = mp.get_logger().info

In [6]:
def init(shared_arr_):
    global shared_arr
    shared_arr = shared_arr_ # must be inherited, not passed as an argument

In [7]:
def tonumpyarray(mp_arr):
    return np.frombuffer(mp_arr.get_obj())

In [8]:
def f(i):
    """synchronized."""
    with shared_arr.get_lock(): # synchronize access
        g(i)

In [9]:
def g(i):
    """no synchronization."""
    info("start %s" % (i,))
    arr = tonumpyarray(shared_arr)
    arr[i] = -1 * arr[i]
    info("end   %s" % (i,))

In [12]:
logger = mp.log_to_stderr()
logger.setLevel(logging.INFO)

# create shared array
N, M = 100, 11
shared_arr = mp.Array(ctypes.c_double, N)
arr = tonumpyarray(shared_arr)

# fill with random values
arr[:] = np.random.uniform(size=N)
arr_orig = arr.copy()

# write to arr from different processes
with closing(mp.Pool(initializer=init, initargs=(shared_arr,))) as p:
    # many processes access the same slice
    stop_f = N // 10
    p.map_async(f, [slice(stop_f)]*M)

    # many processes access different slices of the same array
    assert M % 2 # odd
    step = N // 10
    p.map_async(g, [slice(i, i + step) for i in range(stop_f, N, step)])
p.join()
assert np.allclose(((-1)**M)*tonumpyarray(shared_arr), arr_orig)

[INFO/ForkPoolWorker-5] child process calling self.run()
[INFO/ForkPoolWorker-6] child process calling self.run()
[INFO/ForkPoolWorker-5] child process calling self.run()
[INFO/ForkPoolWorker-7] child process calling self.run()
[INFO/ForkPoolWorker-6] child process calling self.run()
[INFO/ForkPoolWorker-7] child process calling self.run()
[INFO/ForkPoolWorker-8] child process calling self.run()
[INFO/ForkPoolWorker-8] child process calling self.run()
[INFO/ForkPoolWorker-5] start slice(None, 10, None)
[INFO/ForkPoolWorker-5] start slice(None, 10, None)
[INFO/ForkPoolWorker-5] end   slice(None, 10, None)
[INFO/ForkPoolWorker-5] end   slice(None, 10, None)
[INFO/ForkPoolWorker-5] start slice(None, 10, None)
[INFO/ForkPoolWorker-5] start slice(None, 10, None)
[INFO/ForkPoolWorker-5] end   slice(None, 10, None)
[INFO/ForkPoolWorker-5] end   slice(None, 10, None)
[INFO/ForkPoolWorker-6] start slice(None, 10, None)
[INFO/ForkPoolWorker-6] start slice(None, 10, None)
[INFO/ForkPoolWorker-6] 

In [13]:
arr_orig

array([0.81235282, 0.07049956, 0.02825088, 0.4797466 , 0.47677211,
       0.19260485, 0.69172726, 0.64004277, 0.83442437, 0.50813729,
       0.59392831, 0.66794082, 0.51850246, 0.6573046 , 0.72155607,
       0.84347469, 0.65087229, 0.99698912, 0.370602  , 0.95360093,
       0.77154428, 0.29328912, 0.60508574, 0.22717858, 0.13736395,
       0.36327716, 0.53053062, 0.91457096, 0.80032898, 0.89320931,
       0.26307755, 0.4332307 , 0.16362037, 0.37069002, 0.92491644,
       0.61540368, 0.69626279, 0.46966663, 0.50475734, 0.93958009,
       0.512688  , 0.35517291, 0.92269323, 0.77766588, 0.00396624,
       0.53946525, 0.22925286, 0.82863941, 0.23205913, 0.23716827,
       0.01597257, 0.8499676 , 0.13750062, 0.94832543, 0.5938323 ,
       0.48080329, 0.73428766, 0.95581715, 0.21599714, 0.94526026,
       0.99889974, 0.66622197, 0.99345008, 0.63495788, 0.6895107 ,
       0.31927795, 0.21703643, 0.25976639, 0.03844828, 0.43307857,
       0.52127856, 0.45545947, 0.44986354, 0.28187139, 0.35002

4